In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import copy
np.random.seed(101)

In [ ]:
EPSILON = 1e-8

class Gradients:
  def __init__(self, b, c, U, W, V):
    self.b = b
    self.c = c
    self.U = U
    self.W = W
    self.V = V

class RNN():
  def __init__(self):
    self.indexToToken, self.tokenToIndex, self.m, self.eta, self.seq_length, self.K, self.b, self.c, self.U, self.W, self.V, self.text = initialize()
    self.h_history, self.a_history, self.o_history, self.p_history, self.loss_history, self.y_history, h_prev_0 = [], [], [], [], [], [], []
    self.smoothLossValue = None
    
  def forward(self, X, h0, Y):
    if h0 is None:
      h0 = np.zeros((self.m))
    h_prev = h0
    
    self.h_prev_0 = h_prev
    h_history = []
    a_history = []
    p_history = []
    y_history = []

    # Korrekt hittils
    # Transpose X to step through the columns
    index = 0
    for x_t in X.T:
      y_t = Y.T[index]
      a_t = np.dot(self.W, h_prev[:, np.newaxis]) + np.dot(self.U, x_t[:, np.newaxis]) + self.b # Yay
      h_t = tanh(a_t) # Yay
      o_t = np.dot(self.V, h_t) + self.c # Yay (?)
      p_t = softMax(o_t) # Yay
      # loss = self.calcLoss(y_t, p_t) # Todo: save loss history

      h_history.append(h_t.flatten())
      a_history.append(a_t.flatten())
      p_history.append(p_t.flatten())
      y_history.append(y_t.flatten())
      
      h_prev = h_t.flatten()
      index += 1
    
    self.h_history, self.a_history, self.p_history, self.y_history = h_history, a_history, p_history, y_history # Overwrite and save for backpropagation
    loss = self.calcLoss(y_history, p_history)
    if self.smoothLossValue == None:
      self.smoothLossValue = loss
    self.loss_history.append(self.smoothLoss(loss))
  
    return loss
  
  def gethHistory(self):
    h_history_h0 = self.h_history.copy()
    for index, h in enumerate(self.h_history):
      if index == 0:
        h_history_h0[index] = self.h_prev_0
      else:
        h_history_h0[index] = self.h_history[index-1]
    
    return h_history_h0  

  # Perform the backward pass on the RNN
  def backward(self, x_history):
    gradW, gradV, gradU = np.zeros(self.W.shape), np.zeros(self.V.shape), np.zeros(self.U.shape)
    gradb, gradc = np.zeros(self.b.shape), np.zeros(self.c.shape)

    # Claculate all dLdo's
    dlDos = np.zeros((self.seq_length, self.K))
    dlDas = np.zeros((self.seq_length, self.m))
    dlDhs = np.zeros((self.seq_length, self.m))
    index = 0
    for y_t, p_t in zip(self.y_history, self.p_history):
      dLdo_t = -(y_t - p_t) # Yay
      dlDos[index] = dLdo_t
      index += 1
      
    # Init dLda t+1
    dLdh_t = np.dot(dlDos[-1][np.newaxis, :],self.V) # Last dLdo
    dLda_t1 = (dLdh_t.flatten() @ np.diag(1 - np.power(tanh(self.a_history[-1]), 2))) # The last dLDa
    
    dlDas[-1] = dLda_t1
    index = len(self.y_history) - 2
    for _ in range(len(self.y_history)-1):
      dLdo_t = dlDos[index][np.newaxis, :]
      dLdh_t = np.dot(dLdo_t, self.V) + np.dot(dLda_t1, self.W) 
      dLda_t = dLdh_t.flatten() @ np.diag(1 - np.power(tanh(self.a_history[index]), 2))
      dlDas[index] = dLda_t
      index -= 1
      dLda_t1 = dLda_t

    gradW, gradV, gradU, gradb, gradc = np.zeros(self.W.shape), np.zeros(self.V.shape), np.zeros(self.U.shape), np.zeros(self.b.shape), np.zeros(self.c.shape)
    
    h_history_h0 = self.gethHistory() 
    gradU = np.dot(dlDas.T, x_history.T)
    gradW = np.dot(dlDas.T, h_history_h0)
    for t in range(len(self.y_history)):
      gradV += np.dot(dlDos[t].reshape(dlDos[t].shape[0], 1), self.h_history[t].reshape(1, self.h_history[t].shape[0]))
      gradb += dlDas[t][:,np.newaxis]
      gradc += dlDos[t][:,np.newaxis]
    
    return Gradients(np.clip(gradb, -5, 5), np.clip(gradc, -5, 5), np.clip(gradU, -5, 5), np.clip(gradW, -5, 5), np.clip(gradV, -5, 5)) 

  # RNN is a RNN object.
  # h0 is an input vector of the hidden state at t=0.
  # x0 the dummy state at t=0.
  # n is the length of the sequence to be generated.
  def synthesize(self, h0, x0, n):
    h_prev, x_prev = h0, x0
    Y = np.zeros((n, self.K)) # Matrix of One-hot vectors of length n representing letters 
    for i in range(n): # Do a forward pass and sample from the output distribution
      a_t = np.dot(self.W, h_prev[:, np.newaxis]) 
      a_t += np.dot(self.U, x_prev[:, np.newaxis]) 
      a_t += self.b
      h_t = np.tanh(a_t)
      o_t = np.dot(self.V, h_t) + self.c
      p_t = softMax(o_t)
      h_prev = h_t.flatten()
      x_prev = self.genNextX(p_t) # Get a character according to the p_t probability distribution
      Y[i] = x_prev

    return self.matrixOfOneHotToChars(Y)
    
  # p is a Kx1 vector
  def genNextX(self, p):
    rand = random.uniform(0, 1)
    cumProb = 0
    index = 0
    for i, prob in enumerate(p):
      cumProb += prob
      if cumProb >= rand:
        index = i
        break
    
    return charToVec(index, self.K)
    
  def calcLoss(self, y_t, p_t):
    # Calculate the cross entropy loss
    y, p = np.asarray(y_t), np.asarray(p_t)
    return -np.sum(y * np.log(p)) 

  def charSeqToVecSeq(self, charSeq):
    return charSeqToVecSeq(charSeq, self.tokenToIndex, self.K)

  def matrixOfOneHotToChars(self, Y):
    outputSequence = ""
    for K in Y:
      charIndex = np.where(K==1)[0][0]
      outputSequence += self.indexToToken.get(charIndex)
    
    return outputSequence

  def getFullStop(self):
    return charToVec(self.tokenToIndex.get("."), self.K)
  
  def smoothLoss(self, loss):
    toReturn = self.smoothLossValue * 0.999 + loss * 0.001
    self.smoothLossValue = toReturn
    return toReturn

def charToVec(index, K):
  vec = np.zeros(K)
  vec[index] = 1
  return vec

def tanh(a_t):
  return np.tanh(a_t) # This was an in build function? Really? So much time wasted.

def softMax(o_t):
  # Standard definition of the softmax function
  return np.exp(o_t) / np.sum(np.exp(o_t), axis=0)

def read():
  with open("goblet_book.txt") as file:
      text = file.read()
  return text

def initialize():
    text = read()
    # text = text.lower()
    # text = text.replace("\n", " ")
    indexToToken = {}
    n = 0
    for letter in text:
        if letter not in indexToToken.values():
          indexToToken[n] = letter
          n+=1

    tokenToIndex = {v: k for k, v in indexToToken.items()}

    m = 100
    eta = 0.1
    seq_length = 25
    K = len(indexToToken)
    sig = 0.01

    b = np.zeros((m, 1))
    c = np.zeros((K, 1))
    U = np.random.rand(m, K)*sig
    W = np.random.rand(m, m)*sig
    V = np.random.rand(K, m)*sig

    return indexToToken, tokenToIndex, m, eta, seq_length, K, b, c, U, W, V, text

def charSeqToVecSeq(charSeq, tokenToIndex, K):
  veqSeq = np.zeros((K, len(charSeq)))
  for col, char in enumerate(charSeq):
    row = tokenToIndex.get(char) # .lower()
    veqSeq[row,col] = 1
  return veqSeq

def adaGrad(m, gradient, theRnn, param):
  m += np.square(gradient)
  param -= np.divide(theRnn.eta, np.sqrt(m) + EPSILON) * gradient # epsilon does not have to be included in the sqrt
  return m, param
    
def trainRNN():
  rnnH = RNN() # The harry potter rnn :)
  x0 = rnnH.getFullStop() # a "."-character (period, fullstop)
  text = rnnH.text
  mValues = [np.zeros(rnnH.b.shape), np.zeros(rnnH.c.shape), np.zeros(rnnH.U.shape), np.zeros(rnnH.W.shape), np.zeros(rnnH.V.shape)]
  nIter = int(len(text)/25) # Step through the text in chunks of 25 characters
  epochs = 5
  bestRNN = None
  bestLoss = float("inf")
  for epoch in range(epochs):
    h0 = None # The initial hidden state is set to zero. (see forward pass implmentation)
    for i in range(nIter): # ignore the last characters that don't fit into a sequence of length 25
      x_chars = text[i*25:(i+1)*25]
      y_chars = text[i*25+1:(i+1)*25+1]
      X_char_vecs = charSeqToVecSeq(x_chars, rnnH.tokenToIndex, rnnH.K)
      Y_char_vecs = charSeqToVecSeq(y_chars, rnnH.tokenToIndex, rnnH.K)
      
      if epoch == 0 and i == 0:
        print('Not trained: ', rnnH.synthesize(np.zeros((rnnH.m)), x0, 200))
      
      loss = rnnH.forward(X_char_vecs, h0, Y_char_vecs)
      gradients = rnnH.backward(X_char_vecs)
      
      h0 = rnnH.h_history[-1] # The hidden state of the last time step is used as the initial hidden state for the next sequence.
      
      # Update the parameters using AdaGrad
      mValues[0], rnnH.b = adaGrad(mValues[0], gradients.b, rnnH, rnnH.b)
      mValues[1], rnnH.c = adaGrad(mValues[1], gradients.c, rnnH, rnnH.c)
      mValues[2], rnnH.U = adaGrad(mValues[2], gradients.U, rnnH, rnnH.U)
      mValues[3], rnnH.W = adaGrad(mValues[3], gradients.W, rnnH, rnnH.W)
      mValues[4], rnnH.V = adaGrad(mValues[4], gradients.V, rnnH, rnnH.V)
      
      if loss < bestLoss:
        bestLoss = loss
        bestRNN = copy.deepcopy(rnnH)
      
      if ((i + epoch * nIter) % 10000 == 0):
        generatedText = rnnH.synthesize(h0, x0, 200)
        funWords = ['Harry', 'Hermione', 'Weasly', 'Dumbledore', 'Voldemort']
        for word in funWords:
          if word in generatedText:
            print(f'"{word}" is in the generated text!\n{generatedText}\n')
        print(f'Text: (loss: {rnnH.loss_history[-1]}, iteration no. {i + epoch * nIter}, {(1-((i + epoch * nIter)/(5*nIter)))*100}% left)\n{generatedText}\n')
  
  print('Best loss: ' + str(bestLoss))
  return rnnH, bestRNN, x0, None
    


# Develop and debug

In [ ]:
rnnH, bestRNN, x0, h0 = trainRNN()

x = [i for i in range(len(rnnH.loss_history))]
plt.plot(x, rnnH.loss_history)
plt.xlabel("Iteration")
plt.ylabel("Smooth loss")
plt.show()

In [ ]:
bestRNN.synthesize(bestRNN.h_history[-1], x0, 1000)

In [ ]:
# import GradientFunctions

# def debugRNN():
#     rnn = RNN()
#     text = rnn.text
#     X_chars = text[0:25]
#     Y_chars = text[1:26]
#     X_char_vecs = charSeqToVecSeq(X_chars, rnn.tokenToIndex, rnn.K)
#     Y_char_vecs = charSeqToVecSeq(Y_chars, rnn.tokenToIndex, rnn.K)

#     rnn.forward(X_char_vecs, None, Y_char_vecs)
#     grad = rnn.backward(X_char_vecs)

#     rnn.forward(X_char_vecs, None, Y_char_vecs)
#     grad = rnn.backward(X_char_vecs)
#     numgrads = GradientFunctions.numericalGradient(rnn, X_char_vecs, Y_char_vecs, 1e-6, grad)